In [5]:
import os
import asyncio
import nest_asyncio
import requests

from dotenv import load_dotenv
from openai import OpenAI
from openai import AsyncOpenAI


# 加载环境变量
load_dotenv()
if not os.getenv("GITHUB_TOKEN"):
    print("Error: GITHUB_TOKEN is not set in .env file")
    exit(1)

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

# 测试模型列表
BASE_URL = "https://models.github.ai/inference"
headers = {
    "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}

# 初始化 OpenAI 客户端
MODEL_ID = "openai/gpt-4o-mini"


In [6]:
client = OpenAI(api_key=GITHUB_TOKEN, base_url=BASE_URL)

# 调用 LLM
try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": "用中文讲个冷笑话"}],
        temperature=0.2,
    )
    print("LLM response:", response.choices[0].message.content)
except Exception as e:
    print("LLM error:", e)

LLM response: 有一天，一只鸡走进了图书馆，走到图书馆员面前说：“咕咕。”图书馆员给了它三本书。鸡拿着书走了出去。

第二天，鸡又来了，还是说：“咕咕。”图书馆员又给了它三本书。鸡又走了。

第三天，鸡再次来到图书馆，还是说：“咕咕。”图书馆员这次好奇了，决定跟着它。于是他悄悄跟在鸡后面。

鸡走到一个池塘边，把书递给了一只青蛙。青蛙看了看书，摇了摇头，说：“呱，不是这个，不是这个。” 

图书馆员恍然大悟，原来鸡是给青蛙借书呢！


In [ ]:
# Apply nest_asyncio to allow async in Jupyter Notebook
nest_asyncio.apply()

client = AsyncOpenAI(api_key=GITHUB_TOKEN, base_url=BASE_URL)

# Define a list of prompts for batch processing
prompts = [
    "用中文讲一个冷笑话",
    "再讲一个冷笑话，要跟动物有关",
    "讲一个跟程序员有关的冷笑话",
    "讲一个简短的冷笑话",
]

# Asynchronous function to call LLM for a single prompt
async def call_llm(prompt):
    try:
        response = await client.chat.completions.create(
            model=MODEL_ID,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
        return {"prompt": prompt, "response": response.choices[0].message.content}
    except Exception as e:
        return {"prompt": prompt, "response": None, "error": str(e)}


# Run batch calls asynchronously
async def main():
    tasks = [call_llm(prompt) for prompt in prompts]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    for result in results:
        print(f"Prompt: {result['prompt']}")
        if result.get("response"):
            print(f"Response: {result['response']}\n")
        else:
            print(f"Error: {result['error']}\n")
    return results

# Execute async batch calls in Jupyter
results = asyncio.run(main())
print("Batch results:", results)

Prompt: 用中文讲一个冷笑话
Response: 为什么海洋总是那么蓝？

因为鱼在里面游泳的时候，看到水就会说：“哇，真蓝！”

Prompt: 再讲一个冷笑话，要跟动物有关
Error: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'high'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}

Prompt: 讲一个跟程序员有关的冷笑话
Response: 为什么程序员总是喜欢在海边工作？

因为那里有很多“海量数据”！

Prompt: 讲一个简短的冷笑话
Response: 为什么海洋总是很冷？

因为它有很多“海水”（寒水）！

Batch results: [{'prompt': '用中文讲一个冷笑话', 'response': '

In [ ]:
# Importing required libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=GITHUB_TOKEN,
    model=MODEL_ID,
    base_url=BASE_URL,
    temperature=0.2,
)
prompt_template = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])
chain = prompt_template | llm | StrOutputParser()

# Define prompts for batch processing
prompts = [
   "用中文讲一个幽默的笑话",
    "讲一个关于动物的幽默笑话",
    "讲一个与程序员相关的幽默笑话",
    "讲一个简短的幽默笑话",
]

# Asynchronous batch calling with LCEL
async def main():
    results = []
    try:
        responses = await chain.abatch([{"input": prompt} for prompt in prompts])
        results = [{"prompt": prompt, "response": response} for prompt, response in zip(prompts, responses)]
        for result in results:
            print(f"Prompt: {result['prompt']}")
            print(f"Response: {result['response']}\n")
    except Exception as e:
        print("Batch error:", str(e))
        results.append({"prompt": "batch", "response": None, "error": str(e)})
    return results

# Run async batch in Jupyter
results = asyncio.run(main())
print("Batch results:", results)

Prompt: 用中文讲一个幽默的笑话
Response: 有一天，小明去买水果。他看到一个摊位上有个西瓜，特别大，心想：“这个西瓜肯定很甜！”于是他问老板：“这个西瓜多少钱？”

老板说：“20块。”

小明心里一惊：“这么贵啊！我能不能便宜点？”

老板笑着说：“那你要不要试试？如果你觉得甜，我就给你打折。”

小明点点头，老板切了一块给他。小明尝了一口，惊喜地说：“哇，真甜！”

老板笑着说：“那你要不要买一个？”

小明想了想，摇摇头：“算了，我还是买个便宜的苹果吧，西瓜太大了，放不下！”

老板一愣：“你不是说它甜吗？”

小明笑着说：“甜是甜，但我家冰箱不甜啊！”

Prompt: 讲一个关于动物的幽默笑话
Response: 有一天，一只乌龟走进酒吧，点了一杯酒。酒保看着它，问：“你怎么这么慢？”

乌龟回答：“我在等我的朋友们，他们都在后面慢慢爬过来。”

酒保好奇地问：“你朋友是谁？”

乌龟自豪地说：“我有一只兔子朋友和一只蜗牛朋友！”

酒保笑了：“那你们聚会的时候，谁最先到？”

乌龟眨了眨眼：“当然是我，因为我总是提前到达，等他们的时候可以喝酒！”

Prompt: 讲一个与程序员相关的幽默笑话
Response: 当然可以！这里有一个与程序员相关的笑话：

有一天，一个程序员走进酒吧，点了一杯啤酒。酒保问他：“你要什么样的啤酒？”

程序员回答：“随便，反正我只会喝‘0’和‘1’！”

酒保困惑地问：“那你喝得下去吗？”

程序员笑着说：“当然，‘0’是空杯，‘1’是满杯！”

希望这个笑话能让你开心！

Prompt: 讲一个简短的幽默笑话
Response: 有一天，小明问他的朋友：“你知道为什么海洋是蓝色的吗？”

朋友摇摇头。

小明得意地说：“因为鱼都在水里游泳，看到水面反射的天空，就觉得水是蓝色的！”

朋友忍不住笑了：“那你怎么知道鱼不会觉得水是绿色的呢？”

小明想了想，回答：“因为它们没有眼镜！”

Batch results: [{'prompt': '用中文讲一个幽默的笑话', 'response': '有一天，小明去买水果。他看到一个摊位上有个西瓜，特别大，心想：“这个西瓜肯定很甜！”于是他问老板：“这个西瓜多少钱？”\n\n老板说：“20块。”\n\n小明心里一惊：“这么贵啊！我能不能便宜点？”\n\n老板笑着说